In [10]:
import os
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
data = pd.read_csv('POCs/20250322_Airline_Reviews_withImageUrls.csv')
data.shape

(24354, 21)

### Hugging Face - cardiffnlp/twitter-roberta-base-sentiment-latest

In [3]:
from transformers import pipeline
RBSL_PN_MODEL = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment-latest")
RBSL_PN_MODEL('I am happy')

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0


[{'label': 'positive', 'score': 0.9614965319633484}]

In [ ]:

# Make sure reviews are in the right format (list of strings)
review_texts = data['Review'].fillna("").astype(str).tolist()

# Process in batches
batch_size = 32
all_results = []

for i in tqdm(range(0, len(review_texts), batch_size), desc="Processing reviews"):
    batch = review_texts[i:i + batch_size]
    try:
        batch_results = RBSL_PN_MODEL(
            batch,
            truncation=True,
            max_length=512
        )
        all_results.extend(batch_results)
    except Exception as e:
        print(f"Error in batch {i}-{i+batch_size}: {e}")
        all_results.extend([{'label': 'ERROR', 'score': 0.0} for _ in batch])

# Convert to DataFrame
sentiments_df = pd.DataFrame(all_results)
sentiments_df.columns = ['sentiment_label', 'sentiment_scores']